<a href="https://colab.research.google.com/github/jong9810/TensorFlow-2.0/blob/main/10_3_Transfer_Learning_Ex2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Transfer Learning Ex2 : 나만의 이미지 분류 시스템 구현
- 강아지 품종 (치와와, 요크셔테리어, 진돗개, 셰퍼드)]
- 다양한 해상도와 크기를 가지는 이미지 데이터 (노이즈 포함)


### 나만의 이미지로 Training Data 만드는 과정
1. 먼저 dog_image/train 과 같은 임의의 이름을 가지는 루트 디렉토리를 만든 다음, 이러한 루트 디렉토리의 하위 디렉토리로 우리가 분류하고자 하는 이미지의 정답을 나타내는 디렉토리를 생성한다. (hihuahua, jindo_dog, shepherd, yorkshire_terrier와 같은 디렉토리를 생성)
1. 이러한 이미지 정답을 나타내는 각 디렉토리 안에 직접 찍은 사진이나 인터넷에서 다운받은 파일을 최소 100개 이상 저장해 놓는다.
1. ImageDataGenerator.flow_from_directory() 함수를 이용해 데이터를 읽어온다.

In [ ]:
import tensorflow as tf
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

tf.__version__

In [ ]:
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Dense, Dropout, Input
from tensorflow.keras.applications import MobileNet
from tensorflow.keras.layers import Flatten, Conv2D, MaxPooling2D
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive/')

In [ ]:
# 파일 다운로드
import shutil
# 구글 드라이브에 있는 이미지 압축파일을 Colab으로 복사시킴
shutil.copy('/content/gdrive/My Drive/Colab Notebooks/dataset/dog_image.zip', '/content/')

In [ ]:
root_dir = '/content'

import os
import shutil

if os.path.exists(os.path.join(root_dir, 'dog_image')):
    shutil.rmtree(os.path.join(root_dir, 'dog_image'))

In [ ]:
import zipfile

# 압축 파일을 찾아서 압축을 풀어줌
with zipfile.ZipFile(os.path.join(root_dir, 'dog_image.zip'), 'r') as target_file:
    target_file.extractall(os.path.join(root_dir, 'dog_image'))

In [ ]:
import os
import shutil

# test 디렉토리 생성
if not os.path.exists(os.path.join(root_dir, 'dog_image/test')):
    os.mkdir(os.path.join(root_dir, 'dog_image/test'))

# test_image_files 디렉토리 생성
if not os.path.exists(os.path.join(root_dir, 'dog_image/test_image_files')):
    os.mkdir(os.path.join(root_dir, 'dog_image/test_image_files'))

In [ ]:
import os
import glob
import shutil

ratio = 0.1 # train : test = 9 : 1

# src : source (=train), dst : destination (=test) 의 줄임말
src_root_dir = os.path.join(root_dir, 'dog_image/train/')
dst_root_dir = os.path.join(root_dir, 'dog_image/test/')

# 정답(label)을 리스트 형식으로 label_name_list 변수에 저장
label_name_list = os.listdir(src_root_dir)

for label_name in label_name_list: 
    dst_label_name_dir = dst_root_dir + label_name # 정답(label)으로 디렉토리 이름 정의

    # test 디렉토리에 label 디렉토리를 생성
    if not os.path.exists(dst_label_name_dir):
        os.mkdir(dst_label_name_dir)

for label_name in label_name_list:
    # src_root_dir/정답(label) 디렉토리에 있는 파일을 모두 train_image_file_list 변수에 저장
    train_image_file_list = glob.glob(src_root_dir + label_name + '/*')
    split_num = int(ratio * len(train_image_file_list)) # 학습 데이터 이미지 중 테스트 데이터로 옮길 이미지 개수
    test_image_file_list = train_image_file_list[0:split_num]

    # 파일 옮기기 : src_dir -> dst dir
    for image_file in test_image_file_list:
        shutil.move(image_file, dst_root_dir + label_name)

In [ ]:
# test 디렉토리 파일 -> test_image_files 디렉토리로 복사
# 이유 ; 학습을 모두 마친 이후에 테스트하고 싶은 이미지를 랜덤하게 좀더 쉽게 선택하기 위해서
import os
import glob
import shutil

src_root_dir = os.path.join(root_dir, 'dog_image/test/')
dst_root_dir = os.path.join(root_dir, 'dog_image/test_image_files/')

label_name_list = os.listdir(src_root_dir)

for label_name in label_name_list:
    image_file_list = glob.glob(src_root_dir + label_name + '/*')
    print('total [{0}] image file nums => [{1}]'.format(label_name, len(image_file_list)))

    copy_nums = 0

    for image_file in image_file_list:
        shutil.copy(image_file, dst_root_dir) # copy
        copy_nums = copy_nums + 1

    print('total copy nums =>', copy_nums)

In [ ]:
# ImageDataGenerator 정의
IMG_WIDTH = 224
IMG_HEIGHT = 224

train_dir = os.path.join(root_dir, 'dog_image/train/')
validation_dir = os.path.join(root_dir, 'dog_image/train/')
test_dir = os.path.join(root_dir, 'dog_image/test/')

In [ ]:
train_datagen = ImageDataGenerator(rescale=1./255, rotation_range=20, width_shift_range=0.2, height_shift_range=0.2, shear_range=0.2, zoom_range=0.2, validation_split=0.15)
validation_datagen = ImageDataGenerator(rescale=1./255, validation_split=0.15)
# validation_split=0.15 : train 데이터로부터 15%비율로 validation data 생성

In [ ]:
# subset : ImageDataGenerator 객체를 정의할 때, validation_split인자에 값을 넣어주었기 때문에 subset 인자에 값을 넣어주어야 함.
train_generator = train_datagen.flow_from_directory(train_dir, batch_size=16, color_mode='rgb', class_mode='sparse', subset='training', target_size=(IMG_WIDTH, IMG_HEIGHT))
validation_generator = validation_datagen.flow_from_directory(validation_dir, batch_size=16, color_mode='rgb', class_mode='sparse', subset='validation', target_size=(IMG_WIDTH, IMG_HEIGHT))

In [ ]:
print(train_generator.class_indices) # 각각의 정답의 인덱스를 출력

In [ ]:
# 모델 구축(pre-trained MobileNet + 사용자 정의 분류기)
base_model = MobileNet(weights='imagenet', include_top=False, input_shape=(IMG_WIDTH, IMG_HEIGHT, 3))

model = Sequential()

model.add(base_model)

model.add(Flatten())

# User-Defined Classifier
model.add(Dense(32, activation='relu'))
model.add(Dropout(0.25))
model.add(Dense(4, activation='softmax'))

In [ ]:
model.compile(loss='sparse_categorical_crossentropy', optimizer=tf.keras.optimizers.Adam(2e-5), metrics=['accuracy'])
# learning_rate = 2e-5 로 작게 설정 (Transfer Training의 Fine Tuning)
model.summary()

In [ ]:
from tensorflow.keras.callbacks import EarlyStopping

# patience=5 : monitor인자에서 정의한 파라미터가 5번 개선되지 않을 경우 계산을 멈춤
earlystopping = EarlyStopping(monitor='val_loss', patience=5)

hist = model.fit(train_generator, validation_data=validation_generator, epochs=50, callbacks=[earlystopping])

In [ ]:
plt.plot(hist.history['loss'], label='train')
plt.plot(hist.history['val_loss'], label='validation')

plt.title('Loss Trend')
plt.xlabel('epochs')
plt.ylabel('loss')
plt.legend(loc='best')
plt.grid()
plt.show()

In [ ]:
plt.plot(hist.history['accuracy'], label='train')
plt.plot(hist.history['val_accuracy'], label='validation')

plt.title('Accuracy Trend')
plt.xlabel('epochs')
plt.ylabel('accuracy')
plt.legend(loc='best')
plt.grid()
plt.show()

In [ ]:
# 테스트 이미지 파일 생성
import random
import os
import numpy as np
import cv2
import glob

label_dict = {'chihuahua': 0, 'jindo_dog': 1, 'shepherd': 2, 'yorkshire_terrier': 3}

# 테스트 이미지 파일 이름 목록
test_image_files_list = glob.glob(root_dir + '/dog_image/test_image_files/*.jpg')

random.shuffle(test_image_files_list)

# 테스트 할 이미지 파일 이름 : 정답.숫자.jpg
test_num = 16
test_image_files = test_image_files_list[:test_num]

# test_image_files 리스트에서 각 이미지에 대해 정답을 label_list에 저장
label_list = []
for i in range(len(test_image_files)):
    label = test_image_files[i].split('/')[-1].split('.')[0].strip() # .strip() : 문자열의 공백을 제거
    label_list.append(label_dict[label])

# 테스트에 사용할 이미지를 불러와서 src_img_list에 저장
src_img_list = []
for i in range(len(test_image_files)):
    src_img = cv2.imread(test_image_files[i], cv2.IMREAD_COLOR)
    src_img = cv2.resize(src_img, dsize=(IMG_WIDTH, IMG_HEIGHT))
    src_img = cv2.cvtColor(src_img, cv2.COLOR_BGR2RGB)
    src_img = src_img / 255.0

    src_img_list.append(src_img)

# 4차원 텐서 변환
src_img_array = np.array(src_img_list)
label_array = np.array(label_list)

In [ ]:
pred = model.predict(src_img_array)
print(pred.shape)
print(pred)

In [ ]:
class_names = ['chihuahua', 'jindo_dog', 'shepherd', 'yorkshire_terrier']
plt.figure(figsize=(12,12))
for pos in range(len(pred)):
    plt.subplot(4, 4, pos+1)
    plt.axis('off')

    label_str = class_names[label_array[pos]]
    pred_str = class_names[np.argmax(pred[pos])] # 'softmax' 출력이므로 argmax를 통해 인덱스 추출 (softmax는 확률값을 저장)

    plt.title('label : ' + label_str + '\npred : ' + pred_str)

    plt.imshow(src_img_array[pos])

plt.tight_layout()
plt.show()